In [1]:
from src.lightning_modules import FM
from src.networks import StableDiffusionXL, MediumUNet

unet = MediumUNet()
encoder = StableDiffusionXL()

fm = FM(model=unet, encoder_decoder=encoder)

/work3/s214630/Latent-DSB/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded VAE model stabilityai/stable-diffusion-xl-base-1.0


In [2]:
from src.networks import PretrainedMimi
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mimi = PretrainedMimi().to(device)

/work3/s214630/Latent-DSB/.venv/lib/python3.12/site-packages/transformers/models/mimi/modeling_mimi.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Loaded Mimi model


In [11]:
sr = 24000
x = torch.randn(1, 1, int(5.1*sr)).to(device)
y = mimi.encode(x)
print(y.shape)

torch.Size([1, 32, 64])
